In [1]:
import requests
from pprint import pprint
import os



APP_ID = os.environ.get("TFL_APP_ID")
APP_ID = "Halfway"
APP_KEY = os.environ.get("TFL_APP_KEY")
APP_KEY = "21dbb2eeb688456e817278669ba2c9d4"
base_url = "https://api.tfl.gov.uk"


def fetch_tfl_data(endpoint, params=None):
    url = f"{base_url}{endpoint}"
    params = params or {}
    params.update({"app_id": APP_ID, "app_key": APP_KEY})

    try:
        response = requests.get(url, params=params, timeout=15)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        print(f"\nHTTP Error {response.status_code}: {response.text}")
        return None
    except Exception as e:
        print(f"\nError fetching {url}: {str(e)}")
        return None

In [2]:
all_modes = fetch_tfl_data("/Line/Meta/Modes")
for mode in all_modes:
    print(mode["modeName"])


bus
cable-car
coach
cycle
cycle-hire
dlr
elizabeth-line
interchange-keep-sitting
interchange-secure
national-rail
overground
replacement-bus
river-bus
river-tour
taxi
tram
tube
walking


In [3]:
modes = ["tube", "overground", "dlr", "elizabeth-line", "tram"]
lines = fetch_tfl_data("/Line/Mode/" + ",".join(modes))

In [4]:
line_ids = []
for line in lines:
    print(line["id"], line["name"], line["modeName"])
    line_ids.append(line["id"])


bakerloo Bakerloo tube
central Central tube
circle Circle tube
district District tube
dlr DLR dlr
elizabeth Elizabeth line elizabeth-line
hammersmith-city Hammersmith & City tube
jubilee Jubilee tube
liberty Liberty overground
lioness Lioness overground
metropolitan Metropolitan tube
mildmay Mildmay overground
northern Northern tube
piccadilly Piccadilly tube
suffragette Suffragette overground
tram Tram tram
victoria Victoria tube
waterloo-city Waterloo & City tube
weaver Weaver overground
windrush Windrush overground


In [5]:
station_id = {}

for line_id in line_ids:
    stops = fetch_tfl_data(f"/Line/{line_id}/StopPoints?tflOperatedNationalRailStationsOnly=false")
    #print(stops[0].keys())
    #pprint(stops[1], depth=1, compact=True)
    #print("\n\n")
    #pprint(stops[0]["children"])
    for stop in stops:
        name = stop["commonName"]
        naptanId = stop["naptanId"]
        print(f"{naptanId}: {name} - {line_id}")
        station_id[naptanId] = name

940GZZLUBST: Baker Street Underground Station - bakerloo
940GZZLUCHX: Charing Cross Underground Station - bakerloo
940GZZLUEAC: Elephant & Castle Underground Station - bakerloo
940GZZLUEMB: Embankment Underground Station - bakerloo
940GZZLUERB: Edgware Road (Bakerloo) Underground Station - bakerloo
940GZZLUHAW: Harrow & Wealdstone Underground Station - bakerloo
940GZZLUHSN: Harlesden Underground Station - bakerloo
940GZZLUKEN: Kenton Underground Station - bakerloo
940GZZLUKPK: Kilburn Park Underground Station - bakerloo
940GZZLUKSL: Kensal Green Underground Station - bakerloo
940GZZLULBN: Lambeth North Underground Station - bakerloo
940GZZLUMVL: Maida Vale Underground Station - bakerloo
940GZZLUMYB: Marylebone Underground Station - bakerloo
940GZZLUNWY: North Wembley Underground Station - bakerloo
940GZZLUOXC: Oxford Circus Underground Station - bakerloo
940GZZLUPAC: Paddington Underground Station - bakerloo
940GZZLUPCC: Piccadilly Circus Underground Station - bakerloo
940GZZLUQPS: Que

In [48]:
id = "elizabeth"
direction = "outbound"
stops = fetch_tfl_data(f"/Line/{id}/Route/Sequence/{direction}")
pprint(stops, depth=1, compact=True)

{'$type': 'Tfl.Api.Presentation.Entities.RouteSequence, '
          'Tfl.Api.Presentation.Entities',
 'direction': 'outbound',
 'isOutboundOnly': False,
 'lineId': 'elizabeth',
 'lineName': 'Elizabeth line',
 'lineStrings': [...],
 'mode': 'elizabeth-line',
 'orderedLineRoutes': [...],
 'stations': [...],
 'stopPointSequences': [...]}


In [49]:
for station in stops["orderedLineRoutes"][0]["naptanIds"]: # Routes with variations have multiple  entries here
    print(f"{id}: {station}-{station_id[station]}")

elizabeth: 910GABWDXR-Abbey Wood
elizabeth: 910GWOLWXR-Woolwich
elizabeth: 910GCSTMHSXR-Custom House
elizabeth: 910GCANWHRF-Canary Wharf
elizabeth: 910GWCHAPXR-Whitechapel
elizabeth: 910GLIVSTLL-Liverpool Street
elizabeth: 910GFRNDXR-Farringdon
elizabeth: 910GTOTCTRD-Tottenham Court Road
elizabeth: 910GBONDST-Bond Street
elizabeth: 910GPADTLL-Paddington
elizabeth: 910GACTONML-Acton Main Line Rail Station
elizabeth: 910GEALINGB-Ealing Broadway Rail Station
elizabeth: 910GWEALING-West Ealing Rail Station
elizabeth: 910GHANWELL-Hanwell Rail Station
elizabeth: 910GSTHALL-Southall Rail Station
elizabeth: 910GHAYESAH-Hayes & Harlington Rail Station
elizabeth: 910GHTRWAPT-Heathrow Terminals 2 & 3 Rail Station
elizabeth: 910GHTRWTM4-Heathrow Terminal 4 Rail Station


In [62]:
#pprint(stops, depth=1, compact=True)
pprint(stops["orderedLineRoutes"][0], depth=2, compact=True)
route_stops = stops["orderedLineRoutes"][0]["naptanIds"]

{'$type': 'Tfl.Api.Presentation.Entities.OrderedRoute, '
          'Tfl.Api.Presentation.Entities',
 'name': 'Abbey Wood &harr;  Heathrow Terminal 4 ',
 'naptanIds': ['910GABWDXR', '910GWOLWXR', '910GCSTMHSXR', '910GCANWHRF',
               '910GWCHAPXR', '910GLIVSTLL', '910GFRNDXR', '910GTOTCTRD',
               '910GBONDST', '910GPADTLL', '910GACTONML', '910GEALINGB',
               '910GWEALING', '910GHANWELL', '910GSTHALL', '910GHAYESAH',
               '910GHTRWAPT', '910GHTRWTM4'],
 'serviceType': 'Regular'}


In [51]:
id = "elizabeth"
fromStopPointId = "910GPADTLL"
toStopPointId = "910GHTRWTM4"

timetable = fetch_tfl_data(f"/Line/{id}/Timetable/{fromStopPointId}/to/{toStopPointId}")

In [69]:
id = "elizabeth"
from_ = "910GABWDXR"
to_ = "910GWOLWXR"

for from_, to_ in zip(route_stops, route_stops[1:]):
    timetable = fetch_tfl_data(f"/Journey/JourneyResults/{from_}/to/{to_}?useRealTimeLiveArrivals=false")
    durations = []
    for journey in timetable["journeys"]:
        durations.append(journey["duration"])
    print(f"{from_}->{to_}={min(durations)}")





910GABWDXR->910GWOLWXR=3
910GWOLWXR->910GCSTMHSXR=4
910GCSTMHSXR->910GCANWHRF=4
910GCANWHRF->910GWCHAPXR=4
910GWCHAPXR->910GLIVSTLL=3
910GLIVSTLL->910GFRNDXR=2
910GFRNDXR->910GTOTCTRD=2
910GTOTCTRD->910GBONDST=2
910GBONDST->910GPADTLL=4
910GPADTLL->910GACTONML=6
910GACTONML->910GEALINGB=3
910GEALINGB->910GWEALING=3
910GWEALING->910GHANWELL=2
910GHANWELL->910GSTHALL=3
910GSTHALL->910GHAYESAH=3
910GHAYESAH->910GHTRWAPT=7
910GHTRWAPT->910GHTRWTM4=5


In [71]:
id = "elizabeth"
from_ = "910GABWDXR"
to_ = "910GWOLWXR"

for from_, to_ in zip(route_stops, route_stops[1:]):
    timetable = fetch_tfl_data(f"/Journey/JourneyResults/{from_}/to/{to_}?useRealTimeLiveArrivals=false")
    durations = []
    for journey in timetable["journeys"]:
        durations.append(journey["duration"])
    print(f"{from_}->{to_}={min(durations)}")





910GABWDXR->910GWOLWXR=3
910GWOLWXR->910GCSTMHSXR=4
910GCSTMHSXR->910GCANWHRF=4
910GCANWHRF->910GWCHAPXR=4
910GWCHAPXR->910GLIVSTLL=3
910GLIVSTLL->910GFRNDXR=2
910GFRNDXR->910GTOTCTRD=2
910GTOTCTRD->910GBONDST=2
910GBONDST->910GPADTLL=4
910GPADTLL->910GACTONML=6
910GACTONML->910GEALINGB=3
910GEALINGB->910GWEALING=3
910GWEALING->910GHANWELL=2
910GHANWELL->910GSTHALL=3
910GSTHALL->910GHAYESAH=3
910GHAYESAH->910GHTRWAPT=7
910GHTRWAPT->910GHTRWTM4=5


In [59]:
pprint(timetable, depth=3)

{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.ItineraryResult, '
          'Tfl.Api.Presentation.Entities',
 'journeyVector': {'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.JourneyVector, '
                            'Tfl.Api.Presentation.Entities',
                   'from': '1001001',
                   'to': '1002162',
                   'uri': '/journey/journeyresults/910gabwdxr/to/910gwolwxr?app_id=halfway&app_key=21dbb2eeb688456e817278669ba2c9d4',
                   'via': ''},
 'journeys': [{'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Journey, '
                        'Tfl.Api.Presentation.Entities',
               'alternativeRoute': False,
               'arrivalDateTime': '2025-02-06T22:05:00',
               'duration': 3,
               'legs': [...],
               'startDateTime': '2025-02-06T22:02:00'},
              {'$type': 'Tfl.Api.Presentation.Entities.JourneyPlanner.Journey, '
                        'Tfl.Api.Presentation.Entities',


In [54]:
pprint(timetable["timetable"]["routes"][0]["schedules"][0]) # periods/frequency
# https://api-portal.tfl.gov.uk/api-details#api=Line&operation=Line_TimetableByPathFromStopPointIdPathId

IndexError: list index out of range

In [35]:
pprint(timetable["timetable"]["routes"][0]["schedules"][0]["periods"][1]["frequency"]["highestFrequency"]) # periods/frequency
pprint(timetable["timetable"]["routes"][0]["schedules"][0]["periods"][1]["frequency"]["lowestFrequency"]) # periods/frequency

# https://api-portal.tfl.gov.uk/api-details#api=Line&operation=Line_TimetableByPathFromStopPointIdPathId

1.0
5.0


In [ ]:

def time_from_journey(journey):
    return int(journey["hour"]) * 60 + int(journey["minute"])
    
first = timetable["timetable"]["routes"][0]["schedules"][0]["firstJourney"]
first_time = time_from_journey(first)
for schedule in timetable["timetable"]["routes"][0]["schedules"][0]["knownJourneys"][1:]:
    next_time = time_from_journey(schedule)
    print(f"Time between Trains = {next_time - first_time} mins")
    first_time = next_time

    

In [ ]:
pprint(timetable["timetable"]["routes"][0]["stationIntervals"], depth=2)

In [ ]:
for stop in timetable["timetable"]["routes"][0]["stationIntervals"][0]["intervals"]:
    name = station_id[stop["stopId"]]
    arrivalTime = stop["timeToArrival"]
    print(f"{name}: {arrivalTime}") # Why are there 2 for almost all stations?

In [ ]:
for stop in timetable["timetable"]["routes"][0]["stationIntervals"][1]["intervals"]:
    name = station_id[stop["stopId"]]
    arrivalTime = stop["timeToArrival"]
    print(f"{name}: {arrivalTime}") # Why are there 2 for almost all stations?